In [1]:
import dataclasses
import pathlib
import json
import pandas as pd
import os


os.chdir(pathlib.Path.cwd().parents[0])


LEDGER_PATH = "~/Nextcloud/Note/Finanze/ledger/2022.csv"
MMEX_PATH = "~/Nextcloud/Note/Finanze/ledger/2021_mmex.csv"
EXAMPLE_MMEX_PATH = "data/test.csv"
OUTPUT_PATH = "data/test_import.csv"

@dataclasses.dataclass
class LedgerCols:
    DATE: str = "DATE"
    BOH: str = "BOH"
    DESCRIPTION: str = "DESCRIPTION"
    CATEGORY: str = "CATEGORY"
    CURRENCY: str = "CURRENCY"
    AMOUNT: str = "AMOUNT"
    BOH2: str = "BOH2"
    BOH3: str = "BOH3"
    
@dataclasses.dataclass
class MMEXCols:
    Data: str = "Data"
    Conto: str = "Conto"
    Beneficiario: str = "Beneficiario"
    Importo: str = "Importo"
    Valuta: str = "Valuta"
    Categoria: str = "Categoria"
    SottoCategoria: str = "Sotto-Categoria"
    Etichette: str = "Etichette"
    Note: str = "Note"
    Prelievo: str = "Prelievo"
    Deposito: str = "Deposito"
    
ledger_cols = LedgerCols() 
mmex_cols = MMEXCols()

with pathlib.Path("/media/paolo/Kingston SSD/ledger-to-mmex/data/mapped_categories.json").open("r") as f:
    mapped_categories: dict[str, str] = json.load(f)

In [2]:
conto_map: dict[str, str] = {
    "Intesa XME": "Intesa",
    "Contanti Sant'Arcangelo": "Casa"
}

In [3]:
ledger = pd.read_csv(LEDGER_PATH, header=None, names=[x.name for x in dataclasses.fields(LedgerCols())]).drop(columns=[ledger_cols.BOH3, ledger_cols.BOH2, ledger_cols.BOH])
ledger["AMOUNT_NORM"] = abs(ledger["AMOUNT"])
ledger = ledger[ledger["DESCRIPTION"] != "Starting balances"]
ledger = ledger.reset_index(drop=True)

In [5]:
indexes = list(ledger.sort_values(by="DATE").groupby(["DATE", "DESCRIPTION", "AMOUNT_NORM"]).groups.values())
error_idxs = []
for idx in indexes:
    if len(idx) != 2 or abs(idx[0] - idx[1]) != 1:
        print(f"Error: {idx}")
        error_idxs.append(idx)

In [6]:
earn = ledger.loc[indexes[0]]
earn

,DATE,DESCRIPTION,CATEGORY,CURRENCY,AMOUNT,AMOUNT_NORM
0,2022/01/01,Regalo Zia Anna,Assets:Banca:Contanti,€,50.0,50.0
1,2022/01/01,Regalo Zia Anna,Guadagni:Regali,€,-50.0,50.0


In [7]:
spend = ledger.loc[indexes[1]]
spend

,DATE,DESCRIPTION,CATEGORY,CURRENCY,AMOUNT,AMOUNT_NORM
2,2022/01/04,Smartwatch nuovo,Spese:Elettronica:Altro,€,25.4,25.4
3,2022/01/04,Smartwatch nuovo,Assets:Banca:Revolut,€,-25.4,25.4


In [8]:
transfer = ledger.loc[indexes[4]]
transfer

,DATE,DESCRIPTION,CATEGORY,CURRENCY,AMOUNT,AMOUNT_NORM
9,2022/01/07,Trasferimento da Contanti Sant'Arcangelo a Con...,Assets:Banca:Contanti Sant'Arcangelo,€,-400.0,400.0
8,2022/01/07,Trasferimento da Contanti Sant'Arcangelo a Con...,Assets:Banca:Contanti,€,400.0,400.0


## Processo Trasferimento

In [9]:
date = transfer.DATE.tolist()[0]
description = transfer.DESCRIPTION.tolist()[0]
amount = transfer.AMOUNT_NORM.tolist()[0]
currency = "EUR"

account = None
category = "Trasferimento:Trasferimento"
beneficiario = None

for _, row in transfer.sort_values(by="AMOUNT").iterrows():
    if row.AMOUNT < 0:
        account = row.CATEGORY.split(":")[-1]
    if row.AMOUNT > 0:
        beneficiario = row.CATEGORY.split(":")[-1]

print(f"date: {date} \ndescription: {description} \namount: {amount} \ncurrency: {currency} \naccount: {account} \ncategory: {category} \nbeneficiario: {beneficiario}")

date: 2022/01/07 
description: Trasferimento da Contanti Sant'Arcangelo a Contanti 
amount: 400.0 
currency: EUR 
account: Contanti Sant'Arcangelo 
category: Trasferimento:Trasferimento 
beneficiario: Contanti


## Processo Guadagno & Spesa

In [10]:
date = earn.DATE[0]
description = earn.DESCRIPTION[0]
amount = earn.AMOUNT_NORM[0]
currency = "EUR"
account = None
category = None
beneficiario = None
for _, row in earn.iterrows():
    if "Assets" in row.CATEGORY:
        account = row.CATEGORY.split(":")[-1]
    if "Assets" not in row.CATEGORY:
        category = row.CATEGORY

print(f"date: {date} \ndescription: {description} \namount: {amount} \ncurrency: {currency} \naccount: {account} \ncategory: {category} \nbeneficiario: {beneficiario}")

date: 2022/01/01 
description: Regalo Zia Anna 
amount: 50.0 
currency: EUR 
account: Contanti 
category: Guadagni:Regali 
beneficiario: None


In [11]:
date = spend.DATE.tolist()[0]
description = spend.DESCRIPTION.tolist()[0]
amount = spend.AMOUNT_NORM.tolist()[0]
currency = "EUR"
account = None
category = None
beneficiario = None
for _, row in spend.iterrows():
    if "Assets" in row.CATEGORY:
        account = row.CATEGORY.split(":")[-1]
    if "Assets" not in row.CATEGORY:
        category = row.CATEGORY

print(f"date: {date} \ndescription: {description} \namount: {amount} \ncurrency: {currency} \naccount: {account} \ncategory: {category} \nbeneficiario: {beneficiario}")

date: 2022/01/04 
description: Smartwatch nuovo 
amount: 25.4 
currency: EUR 
account: Revolut 
category: Spese:Elettronica:Altro 
beneficiario: None


In [55]:
indices = ledger.groupby([ledger_cols.DATE, ledger_cols.DESCRIPTION, "AMOUNT_NORM"]).indices.values()

def detect_transaction_type(accounts: list[str]) -> str:
    for acc in [acc.split(':')[0].lower() for acc in accounts]:
        if acc == "guadagni":
            return "Deposit"
        if acc == "spese":
            return "Withdrawal"
    return "Transfer"

def extract_category(accounts: list[str], transaction_type: str) -> tuple[str, str]:
    if transaction_type == "Transfer":
        return "Trasferimento", "Trasferimento"
    
    for account in accounts:
        if "Assets" in account:
            continue
        
        splits: list[str] = mapped_categories[account].split(":")
        if len(splits) == 1:
            return splits[0], ""
        return ":".join(splits[:-1]), splits[-1]
        
def extract_transaction_accounts_payee(transaction: pd.DataFrame) -> tuple[str, str, None]:
    from_account, to_account, payee = None, None, None
    
    for _, row in transaction.iterrows():
        if "Assets" in row[ledger_cols.CATEGORY] and not from_account:
            from_account = row[ledger_cols.CATEGORY].split(':')[-1]
    return from_account, to_account, payee

def extract_transfer_transaction_account_payee(transaction: pd.DataFrame) -> tuple[str, str, None]:
    from_account, to_account, payee = None, None, None
    
    for _, row in transaction.iterrows():
        if row[ledger_cols.AMOUNT] < 0 and not from_account:
            from_account = row[ledger_cols.CATEGORY].split(':')[-1]
        if row[ledger_cols.AMOUNT] > 0 and not to_account:
            to_account = row[ledger_cols.CATEGORY].split(':')[-1]
    
    return from_account, to_account, payee

def process_transaction(dataframe: pd.DataFrame):
    date: str = dataframe[ledger_cols.DATE][0].replace("/", "-")
    transaction_type: str = detect_transaction_type(dataframe[ledger_cols.CATEGORY].tolist())
    currency: str = "EUR" #dataframe[ledger_cols.CURRENCY][0]
    category, sub_category = extract_category(dataframe[ledger_cols.CATEGORY].tolist(), transaction_type)
    amount: float = dataframe["AMOUNT_NORM"][0]
    note: str = dataframe[ledger_cols.DESCRIPTION][0]

    from_account: str = ""
    to_account: str = ""
    payee: str = ""
    
    if transaction_type == "Transfer":
        from_account, to_account, payee = extract_transfer_transaction_account_payee(dataframe[[ledger_cols.CATEGORY, ledger_cols.AMOUNT]])
    else:
        from_account, to_account, payee = extract_transaction_accounts_payee(dataframe[[ledger_cols.CATEGORY]])

    
    
    return [date, "R", transaction_type, conto_map.get(from_account, from_account), conto_map.get(to_account, to_account), payee, amount, currency, category, sub_category, note]

processed_transactions = []
for idx in indices:
    if len(idx) != 2:
        continue
    processed_transactions.append(process_transaction(ledger.loc[idx].reset_index(drop=True)))


In [56]:
mmex_test = pd.read_csv(EXAMPLE_MMEX_PATH)
mmex_test.insert(1, "Stato", "R")
mmex_test.insert(4, "ToConto", "")

print(mmex_test.columns)
processed_dataframe = pd.DataFrame(processed_transactions, columns=mmex_test.columns) #.to_csv(OUTPUT_PATH, index=False)
processed_dataframe.value_counts("ToConto")

Index(['Data', 'Stato', 'Tipo', 'Conto', 'ToConto', 'Beneficiario', 'Importo',
       'Valuta', 'Categoria', 'Sotto-Categoria', 'Note'],
      dtype='object')


ToConto
Revolut               68
Contanti              42
Intesa                 6
PAC EU                 2
PAC Internazionale     2
Paypal                 1
Name: count, dtype: int64

In [61]:
processed_dataframe.query("Conto == 'Intesa' & Categoria != 'Trasferimento'").sort_values("Data", ascending=False).head(10)

,Data,Stato,Tipo,Conto,ToConto,Beneficiario,Importo,Valuta,Categoria,Sotto-Categoria,Note
536,2022-12-31,R,Withdrawal,Intesa,None,None,2.90,EUR,Tasse,,Imposta di bollo
528,2022-12-25,R,Withdrawal,Intesa,None,None,4.99,EUR,Informatica,Software,Spotify
527,2022-12-20,R,Withdrawal,Intesa,None,None,25.00,EUR,Altre Uscite,,Donazione anziani
526,2022-12-20,R,Withdrawal,Intesa,None,None,8.10,EUR,Viaggi,Trasporti,Biglietto sita Salerno san brancato
525,2022-12-20,R,Withdrawal,Intesa,None,None,2.60,EUR,Viaggi,Trasporti,Biglietto salerno fisciano a/r
524,2022-12-20,R,Withdrawal,Intesa,None,None,5.10,EUR,Viaggi,Trasporti,Biglietto Napoli Salerno
520,2022-12-19,R,Withdrawal,Intesa,None,None,12.46,EUR,Viaggi,Cibo,Pizza Varsavia
518,2022-12-19,R,Withdrawal,Intesa,None,None,6.05,EUR,Viaggi,Trasporti,Biglietto 24 ore Varsavia
519,2022-12-19,R,Withdrawal,Intesa,None,None,4.73,EUR,Viaggi,Cibo,Colazione Varsavia
521,2022-12-19,R,Withdrawal,Intesa,None,None,16.18,EUR,Viaggi,Altro,Prelievo Varsavia porcodio


In [18]:
import re

def convert_conto_name(old_conto: str) -> str:
    for k in conto_map:
        if k in old_conto:
            return re.sub(k, conto_map[k], old_conto)

    return old_conto

processed_dataframe["Beneficiario"] = processed_dataframe["Beneficiario"].apply(convert_conto_name)
processed_dataframe["Conto"] = processed_dataframe["Conto"].apply(convert_conto_name)
processed_dataframe["Note"] = processed_dataframe["Note"].apply(convert_conto_name)

In [19]:
processed_dataframe

,Data,Stato,Tipo,Conto,Beneficiario,Importo,Valuta,Categoria,Sotto-Categoria,Note
0,2022-01-01,R,Deposit,Contanti,None,50.00,EUR,Entrate,Regali,Regalo Zia Anna
1,2022-01-04,R,Withdrawal,Revolut,None,-25.40,EUR,Informatica,Altro,Smartwatch nuovo
2,2022-01-04,R,Withdrawal,Contanti,None,-10.00,EUR,Cibo,Mangiare fuori,StikHouse Anna Marù
3,2022-01-05,R,Withdrawal,Contanti,None,-4.00,EUR,Cibo:Mangiare fuori,Colazione,Colazione Marù
4,2022-01-07,R,Transfer,Casa,> Contanti,400.00,EUR,Trasferimento,Trasferimento,Trasferimento da Casa a Contanti
...,...,...,...,...,...,...,...,...,...,...
532,2022-12-26,R,Transfer,Intesa,> Contanti,26.98,EUR,Trasferimento,Trasferimento,Trasferimento da Intesa a Contanti
533,2022-12-27,R,Deposit,Casa,None,100.00,EUR,Entrate,Regali,Regalo Mamma
534,2022-12-29,R,Deposit,Contanti,None,51.00,EUR,Altre Entrate,,Soldi che erano nel portafogli che mi ha regal...
535,2022-12-30,R,Withdrawal,Contanti,None,-1.50,EUR,Regali,,libro claudia mafia
